In [127]:
import pandas as pd
import numpy as np
import os

os.chdir("/Users/rahulprakash/Desktop/Datathon")  # Replace with your actual path

scoring = pd.read_csv("scoring.csv")
training = pd.read_csv("training.csv")

In [145]:
training['Model Year Difference'] = training['Date'] - training['Model Year']
training

,Date,Vehicle Category,GVWR Class,Fuel Type,Model Year,Fuel Technology,Electric Mile Range,Number of Vehicles Registered at the Same Address,Region,Vehicle Population,Model Year Difference
0,2019,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,≥4,Statewide,395883,-1.0
1,2020,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,1,Statewide,370954,0.0
2,2021,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,1,Statewide,349406,1.0
3,2019,P,Not Applicable,Gasoline,2019.0,ICE,Not Applicable,≥4,Statewide,348475,0.0
4,2019,P,Not Applicable,Gasoline,2018.0,ICE,Not Applicable,≥4,Statewide,333296,1.0
...,...,...,...,...,...,...,...,...,...,...,...
41048,2019,B,Not Applicable,Diesel,1983.0,ICE,Not Applicable,1,Statewide,1,36.0
41049,2019,B,Not Applicable,Diesel,1980.0,ICE,Not Applicable,2,Statewide,1,39.0
41050,2019,B,Not Applicable,Diesel,1978.0,ICE,Not Applicable,3,Statewide,1,41.0
41051,2019,B,Not Applicable,Diesel,1976.0,ICE,Not Applicable,2,Statewide,1,43.0


In [195]:
#Needs Imputation

print((scoring == 'Unknown').sum())
print(training.isna().sum())

Date                                                    0
Vehicle Category                                        0
GVWR Class                                           1108
Fuel Type                                               0
Model Year                                              0
Fuel Technology                                         0
Electric Mile Range                                   505
Number of Vehicles Registered at the Same Address       0
Region                                                  0
Vehicle Population                                      0
dtype: int64
Date                                                 0
Vehicle Category                                     0
GVWR Class                                           0
Fuel Type                                            0
Model Year                                           0
Fuel Technology                                      0
Electric Mile Range                                  0
Number of Vehicles Reg

In [129]:
#Impute GVWR Class
import scipy.stats as stats

def cramers_v(chi2, n, dof):
    return np.sqrt(chi2 / (n * min(dof, 1)))  # Avoids division by zero

contingency_table = pd.crosstab(training['GVWR Class'], training['Number of Vehicles Registered at the Same Address'])
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
cramers_v_value = cramers_v(chi2, training.shape[0], dof)
cramers_v_value

0.05444336700479962

In [130]:
counts = training.groupby('GVWR Class')['Vehicle Category'].value_counts().unstack()
counts

Vehicle Category,B,BS,BT,MC,MH,P,T1,T2,T3,T4,T5,T6,T7
GVWR Class,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,1361.0,1782.0,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2177.0,2060.0,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1778.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1593.0,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1615.0,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1961.0,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1908.0,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1854.0
Not Applicable,2306.0,2175.0,1922.0,2038.0,1961.0,3800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
training.loc[(training['GVWR Class'] == 'Unknown') & (training['Vehicle Category'] == 'T1'), 'GVWR Class'] = '1'
training.loc[(training['GVWR Class'] == 'Unknown') & (training['Vehicle Category'] == 'T2'), 'GVWR Class'] = '1'
training.loc[(training['GVWR Class'] == 'Unknown') & (training['Vehicle Category'] == 'T3'), 'GVWR Class'] = '2'
training.loc[(training['GVWR Class'] == 'Unknown') & (training['Vehicle Category'] == 'T4'), 'GVWR Class'] = '2'
training.loc[(training['GVWR Class'] == 'Unknown') & (training['Vehicle Category'] == 'T5'), 'GVWR Class'] = '3'
training.loc[(training['GVWR Class'] == 'Unknown') & (training['Vehicle Category'] == 'T7'), 'GVWR Class'] = '8'



# Define probability distribution and possible values
possible_values = ['4', '5', '6', '7'] # Example GVWR Class values
probabilities = [0.25, 0.25, 0.25, 0.25]  # Probabilities for each value
indices = training.loc[(training['GVWR Class'] == 'Unknown') & (training['Vehicle Category'] == 'T6')].index
training.loc[indices, 'GVWR Class'] = np.random.choice(possible_values, size=len(indices), p=probabilities)




In [132]:
contingency_table = pd.crosstab(training['Electric Mile Range'], training['GVWR Class'])
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
cramers_v_value = cramers_v(chi2, training.shape[0], dof)
cramers_v_value

0.23251641057258457

In [133]:
counts = training.groupby('Fuel Technology')['Electric Mile Range'].value_counts().unstack()
counts

Electric Mile Range,0 to 50 miles,101 to 150 miles,51 to 100 miles,>150 miles,Not Applicable,Unknown
Fuel Technology,,,,,,
BEV,11.0,162.0,197.0,188.0,NaN,2802.0
FCEV,NaN,NaN,4.0,88.0,NaN,33.0
ICE,NaN,NaN,NaN,NaN,37098.0,NaN
PHEV,364.0,NaN,48.0,NaN,NaN,58.0


In [134]:
counts = training.groupby(['Fuel Type', 'Fuel Technology'])['Electric Mile Range'].value_counts().unstack()
counts.head(50)

Electric Mile Range          0 to 50 miles  101 to 150 miles  51 to 100 miles  \
Fuel Type   Fuel Technology                                                     
Diesel      BEV                        NaN               NaN              1.0   
            ICE                        NaN               NaN              NaN   
Electric    BEV                       11.0             162.0            178.0   
Gasoline    BEV                        NaN               NaN             18.0   
            ICE                        NaN               NaN              NaN   
            PHEV                     364.0               NaN             48.0   
Hydrogen    FCEV                       NaN               NaN              4.0   
Natural Gas ICE                        NaN               NaN              NaN   
Unknown     ICE                        NaN               NaN              NaN   

Electric Mile Range          >150 miles  Not Applicable  Unknown  
Fuel Type   Fuel Technology                                       
Diesel      BEV                     NaN             NaN      1.0  
            ICE                     NaN         15633.0      NaN  
Electric    BEV                   188.0             NaN   2780.0  
Gasoline    BEV                     NaN             NaN     21.0  
            ICE                     NaN         17879.0      NaN  
            PHEV                    NaN             NaN     58.0  
Hydrogen    FCEV                   88.0             NaN     33.0  
Natural Gas ICE                     NaN          3500.0      NaN  
Unknown     ICE                     NaN            86.0      NaN

In [135]:
training.loc[(training['Electric Mile Range'] == 'Unknown') & (training['Fuel Type'] == 'Diesel') & (training['Fuel Technology'] == 'BEV'), 'Electric Mile Range'] = '51 to 100 miles'
training.loc[(training['Electric Mile Range'] == 'Unknown') & (training['Fuel Type'] == 'Gasoline') & (training['Fuel Technology'] == 'BEV'), 'Electric Mile Range'] = '51 to 100 miles'
training.loc[(training['Electric Mile Range'] == 'Unknown') & (training['Fuel Type'] == 'Hydrogen') & (training['Fuel Technology'] == 'FCEV'), 'Electric Mile Range'] = '>150 miles'



#Electric + BEV
possible_values = ["0 to 50 miles", "101 to 150 miles", "51 to 100 miles", ">150 miles"] # Example GVWR Class values
probabilities = [11/539, 162/539, 178/539, 188/539]  # Probabilities for each value
indices = training.loc[(training['Electric Mile Range'] == 'Unknown') & (training['Fuel Type'] == 'Electric') & (training['Fuel Technology'] == 'BEV')].index
training.loc[indices, 'Electric Mile Range'] = np.random.choice(possible_values, size=len(indices), p=probabilities)

#Gas + PHEV
possible_values = ["0 to 50 miles", "51 to 100 miles"] # Example GVWR Class values
probabilities = [364/412, 48/412]  # Probabilities for each value
indices = training.loc[(training['Electric Mile Range'] == 'Unknown') & (training['Fuel Type'] == 'Gasoline') & (training['Fuel Technology'] == 'PHEV')].index
training.loc[indices, 'Electric Mile Range'] = np.random.choice(possible_values, size=len(indices), p=probabilities)


In [194]:
contingency_table = pd.crosstab(training['Fuel Type'], training['Electric Mile Range'])
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
cramers_v_value = cramers_v(chi2, training.shape[0], dof)
cramers_v_value

1.0083244866452465

In [187]:
# Define population bins and labels
bins = [0, 50000, 100000, 150000, 200000, 250000, 300000, 350000, 400000, np.inf]  # Define range boundaries
labels = [0, 1, 2, 3, 4, 5, 6, 7, 8]  # Assign a category number to each range

# Create the 'Population Category' column
training['Population Category'] = pd.cut(training['Vehicle Population'], bins=bins, labels=labels, right=False)
training

,Date,Vehicle Category,GVWR Class,Fuel Type,Model Year,Fuel Technology,Electric Mile Range,Number of Vehicles Registered at the Same Address,Region,Vehicle Population,Model Year Difference,Population Category
0,2019,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,≥4,Statewide,395883,-1.0,7
1,2020,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,1,Statewide,370954,0.0,7
2,2021,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,1,Statewide,349406,1.0,6
3,2019,P,Not Applicable,Gasoline,2019.0,ICE,Not Applicable,≥4,Statewide,348475,0.0,6
4,2019,P,Not Applicable,Gasoline,2018.0,ICE,Not Applicable,≥4,Statewide,333296,1.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...
41048,2019,B,Not Applicable,Diesel,1983.0,ICE,Not Applicable,1,Statewide,1,36.0,0
41049,2019,B,Not Applicable,Diesel,1980.0,ICE,Not Applicable,2,Statewide,1,39.0,0
41050,2019,B,Not Applicable,Diesel,1978.0,ICE,Not Applicable,3,Statewide,1,41.0,0
41051,2019,B,Not Applicable,Diesel,1976.0,ICE,Not Applicable,2,Statewide,1,43.0,0


In [188]:

# Compute the median 'Model Year Difference' for each 'Vehicle Category'
median_model_year_diff = training.groupby('Vehicle Category')['Model Year Difference'].median()

# Fill NaN values in 'Model Year Difference' using the median from the corresponding 'Vehicle Category'
training['Model Year Difference'] = training.apply(
    lambda row: median_model_year_diff[row['Vehicle Category']] if pd.isna(row['Model Year Difference']) else row['Model Year Difference'],
    axis=1
)

# Verify that there are no more NaN values in 'Model Year Difference'
training['Model Year Difference'].isna().sum()

0

In [190]:
training.loc[training['Model Year'].isna(), 'Model Year'] = (
    training['Date'] - training['Model Year Difference']
)